## Section 7.3. The Sources of the Welfare Gains from Trade—A Quantitative Decomposition

Main file to compute the local decomposition of welfare into different components. Organized in the following way:

- [Computational Preliminaries](#comp)

- [Welfare Decomposition](#decomp)

- [How Inefficient is the Decentralized Equilibrium?](#how)

- [Comparison to ACR](#acr)


---
### <a name="comp"></a> Computational Preliminaries

Basicly get the package setup and settins for the total dervitive compuations.

In [52]:
using PerlaTonettiWaugh
using CSV

In [29]:
params = load_parameters("parameters/calibration_params.csv") 
# Read in the baseline calibraiton parameters

(ρ = 0.0203380446685169, σ = 3.16692413583811, θ = 4.98897658793826, κ = 0.104196324793307, χ = 0.126846612050694, μ = -0.0310646242175711, υ = 0.0483011406016648, ζ = 1, δ = 0.02, N = 10, γ = 1, η = 0, Theta = 1, d_0 = 3.0224928254626, d_T = 2.82024354291634)

In [30]:
# settins for numerically computing the derivative
d_baseline = params.d_0
ϵ = 0.000001
d_ϵ = d_baseline + ϵ;

#### Baseline Steady State

Just to confirm where we are starting from, i.e. growth and trade should line up with the values that the model was targeted to match.

In [31]:
params_baseline_1 = merge(params, (d = params.d_0,))
baseline_sol = stationary_algebraic(params_baseline_1, settings_defaults());

In [32]:
println("-------------------------------------------------------------------------")
println("")
println("Baseline Steady State")
println("Growth in Baseline: ", 100*baseline_sol.g)
println("Imports/GDP in Baseline: ", 100*(1 - baseline_sol.λ_ii))
println("Normalized Consumption in Baseline: ", 100*baseline_sol.c)

-------------------------------------------------------------------------

Baseline Steady State
Growth in Baseline: 0.7913401963163271
Imports/GDP in Baseline: 10.629127170507878
Normalized Consumption in Baseline: 87.06834939103561


#### Counterfactual Steady State

Then let's change trade costs by a very small amount...

In [33]:
params_counterfactual_1 = merge(params, (d = d_ϵ,))
sol = stationary_algebraic(params_counterfactual_1, settings_defaults());

In [34]:
println("-------------------------------------------------------------------------")
println("")
println("Small change in trade costs")
println("Growth: ", 100*sol.g)
println("Imports/GDP: ", 100*(1 - sol.λ_ii))
println("Normalized Consumption: ", 100*sol.c)

-------------------------------------------------------------------------

Small change in trade costs
Growth: 0.7913392605706997
Imports/GDP: 10.629111302522997
Normalized Consumption: 87.06835265233951


#### Total Derivative

Now we will compute the total derivative using the function ``total_derivative``

In [35]:
tot_d = total_derivative(params_baseline_1, ϵ);

---
### <a name="decomp"></a> Welfare Decomposition

So the function above, started from the equilibrium steady state allocation of the calibrated model, and compute the change in welfare from a very small reduction in trade costs. The object ``tot_d`` contains a bunch of information about the derivative (e.g. checks, the level of the derivatives, different terms). Below we present the contribution of each of the direct and indirect terms as a percentage of the total change.

The presentaiton below mimics exactly equation (60) of the March 2020 NBER WP version.

#### Direct effect on consumption: $\bar{U}_{1} \frac{\partial f_c}{\partial d}$

In [36]:
println("Direct effect on consumption (in percent): ", round(100*tot_d.U1_partial_fc_d_frac,digits = 3))

Direct effect on consumption (in percent): 8.321


#### Indirect effect on variety: $\bar{U}_{1} \frac{\partial f_c}{\partial \Omega} \frac{\mathrm{d} f_{\Omega} }{\mathrm{d} d}$

In [37]:
println("Indirect effect on consumption (in percent): ", round(100*tot_d.U1_decomp_fc_Omega_Omega_d_frac,digits = 3))

Indirect effect on consumption (in percent): -7.279


#### Indirect effect on exporter entry: $\bar{U}_{1} \frac{\partial f_c}{\partial \hat{z}} \frac{\mathrm{d}  f_{\hat{z}} }{\mathrm{d} d}$

In [38]:
println("Indirect effect on exporter entry (in percent): ", round(100*tot_d.U1_decomp_fc_zhat_zhat_d_frac,digits = 3))

Indirect effect on exporter entry (in percent): 0.0


#### Indirect effect on growth: $\bar{U}_{1} \frac{\partial f_c}{\partial g} \frac{\mathrm{d} f_{g} }{\mathrm{d} d}$

In [39]:
println("Indirect effect on growth (in percent): ", round(100*tot_d.U1_decomp_fc_g_g_d_frac,digits = 3))

Indirect effect on growth (in percent): -9.904


#### Direct effect on growth: $\bar{U}_{2} \frac{\mathrm{d} f_g}{\mathrm{d} d}$

In [40]:
println("Indirect effect on growth (in percent): ", round(100*tot_d.U2_decomp_g_d_frac,digits = 3))

Indirect effect on growth (in percent): 108.863


#### Commentary (from paper)

First note that the direct effect of a change in trade costs on the consumption level accounts for 8.32 percent of the gains from trade.The next three components are the indirect effects of trade costs on consumption through changes in variety, the exporter threshold, and growth. The indirect effects on consumption of changes in variety (-7.28%) and growth (-9.90%) both reduce the gains from trade. The exporter margin effect is zero. The final term represents the main source of the gains from trade—the direct effect on growth (108.86%)

---
### <a name="how"></a> How Inefficient is the Decentralized Equilibrium?

Here is the issue: The indirect effect and direct effect on growth can be rearranged as:

$\left[ \bar{U}_{1} \frac{\partial f_c}{\partial g} + \bar{U}_{2}\right]  \frac{\mathrm{d} f_{g} }{\mathrm{d} d}$

and the results above show that it contributes about 100 percent to the welfare gains from trade. This term could be large for two reasons: (i) the economy is very inefficient relative to the direct consumption effect or (ii) because the economy has a large elasticity
of growth to trade costs. Which is it?

To facilate understanding, we put eveything in consumption units.

#### Inefficincy term: $\rho \left[\bar{U}_{1} \frac{\partial f_c}{\partial g} + \bar{U}_{2}\right]$

In [68]:
println("Inefficincy term (CE units, percent of increase): ", round(tot_d.planner_0_g_CE,digits = 3))

Inefficincy term (CE units, percent of increase): 44.696


A useful benchmark to discuss the size of this inefficiency is a comparison with an exogenous increase in growth that does not require resources to be drawn away from other activities. In steady state, an exogenous increase in growth of $\mathrm{d}g$—leaving the level of consumption unchanged—would increase consumption equivalent utility by $\frac{\mathrm{d}g}{\rho}$

#### Free Growth: $\frac{\mathrm{d}g}{\rho}$

In [42]:
println("Inefficincy term (CE units, percent increase): ", round(tot_d.MU_g_CE,digits = 3))

Inefficincy term (CE units, percent increase): 49.169


What this means is a one percentage point increase in “free” growth generates a 49.2 percent increase in consumption equivalent welfare

#### Semi-Elasticity of Growth: $\frac{\mathrm{d} f_{g} }{\mathrm{d} d}$

Another reason the total gains may be large is not because of the inefficiency, but that growth is very sensetive to changes in trade costs. In the model, the semi-elasticity is

In [43]:
println("Semi-Elasticity of Growth: ", round(100*tot_d.semi_elasticity_g_d ,digits = 3))

Semi-Elasticity of Growth: -2.828


Extrapolating using this local elasticity, the ten percent reduction in trade costs that we explore in Section 7-4 increases growth by approximately 0.19 percentage points. We do not have strong evidence regarding the plausibility of this semi-elasticity value. As a point of comparison, we compute the average productivity growth for non-overlapping decades in the U.S. from 1950-2017. The standard deviation of those decadal averages is 0.78. 

The notebook ``[section_7-1.ipynb](section_7-1.ipynb)`` computes this value and saves it. Below we readin the saved value for completness...

In [67]:
temp = CSV.read("./src/calibration/data/dec_std_avg.csv", datarow=1);
println("Standard deviation of decadal average growth: ", round(temp[1, 1], digits = 2))

Standard deviation of decadal average growth: 0.78


Thus, movements in the growth rate that we find and study are well within the range of historical variation in the long-run aggregate productivity growth rate.

---
### <a name="acr"></a> ACR Comparision

Here we show that quantiativly, the formula of ACR applied in our framework closely corresponds to the direct effect on consumption $\left(\rho \bar{U}_{1} \frac{\partial f_c}{\partial d}\right)$ expressed in consumption units. What we do is compare this versus the ACR formula and express it as a ratio.

In [44]:
println("Direct Consumption effect as percent of ACR: ", round(100*tot_d.U1_partial_fc_d_ACRfrac,digits = 2))

Direct Consumption effect as percent of ACR: 98.81
